In [1]:
# DBMS term project - Howard and Shawn 
# Date - 10/15/2019 
# Goal - This notebook will ingest images in HDF5 to Keras/TF-based classification 
# The work is based on the official Keras example https://keras.io/examples/cifar10_cnn/

In [2]:
#import keras 
import tensorflow 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
import os

In [3]:
#from keras.datasets import cifar10
from tensorflow.keras import optimizers
import h5py
import numpy as np 
from sklearn.model_selection import train_test_split


In [4]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20 
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [5]:
# load image and label data from hdf5 as NP array 
with h5py.File('./data/hdf5_20000_images.h5','r') as f:
    x_data = np.array(f['images'])
    y_data = np.array(f['meta'])

In [6]:
# split train /test data
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

In [7]:
# The data, split between train and test sets:
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (16000, 32, 32, 3)
16000 train samples
4000 test samples


In [8]:
# Convert class vectors to binary class matrices.
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)


In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
# initiate RMSprop optimizer
opt = tensorflow.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

In [11]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [12]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test, y_test),
              shuffle=True)


Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 50s 3ms/sample - loss: 2.0380 - acc: 0.2463 - val_loss: 1.8486 - val_acc: 0.3358
Epoch 2/10
16000/16000 [==============================] - 50s 3ms/sample - loss: 1.7569 - acc: 0.3622 - val_loss: 1.6640 - val_acc: 0.3940
Epoch 3/10
16000/16000 [==============================] - 49s 3ms/sample - loss: 1.6456 - acc: 0.4069 - val_loss: 1.5562 - val_acc: 0.4372
Epoch 4/10
 9376/16000 [================>.............] - ETA: 18s - loss: 1.5792 - acc: 0.4282

KeyboardInterrupt: 

In [0]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [0]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [0]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))



In [0]:
# confirm TF sees the GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())



In [0]:
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0